## Data preprocessing

In [ ]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(ggpubr)

In [ ]:
# Lecture des données
path <- "data/"
song <- read.csv(paste0(path, "spotify_songs.csv"), header = TRUE, sep = ",")

# Premières lignes du jeu de données
head(song)

# Vérification du contenu
summary(song)

In [ ]:
# Check the data types
str(song)

In [ ]:
# Drop the track_id, track_album_id, playlist_id columns
song <- song[, -c(1, 5, 9)]

# As factor the categorical variables track_artist, playlist_genre, playlist_subgenre, key, mode, playlist_name, track_album_name
song$playlist_name <- as.factor(song$playlist_name)
song$track_album_name <- as.factor(song$track_album_name)
song$track_artist <- as.factor(song$track_artist)
song$playlist_genre <- as.factor(song$playlist_genre)
song$playlist_subgenre <- as.factor(song$playlist_subgenre)
song$key <- factor(song$key, levels = c(-1, 0:11), labels = c("No key detected", "C", "C♯/D♭", "D", "D♯/E♭", "E", "F", "F♯/G♭", "G", "G♯/A♭", "A", "A♯/B♭", "B"))
song$mode <- factor(song$mode, levels = c(0, 1), labels = c("minor", "major"))

# track_album_release_date to date (if the full date is not available, we will use the first day of the year)
song$track_album_release_date <- as.Date(ifelse(nchar(song$track_album_release_date) != 10, 
                                                paste0(substr(song$track_album_release_date, 1, 4), "-01-01"), 
                                                song$track_album_release_date), 
                                         format = "%Y-%m-%d")

# Convert the duration_ms to seconds and rename the column to duration_s
song$duration_s <- song$duration_ms / 1000
song$duration_ms <- NULL

# Check the modified dataset
summary(song)
head(song)

In [ ]:
# Check for missing values
colSums(is.na(song))

# Drop the missing values
song <- na.omit(song)

# Check the modified dataset
colSums(is.na(song))

str(song)

## 2. Réduction de dimension par Analyse en Composantes Principales (ACP)

Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure. Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

### 2.1. Format des données

Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (playlist_genre) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données avant de faire l'ACP car l'ACP est sensible à l'échelle des variables. On va donc centrer et réduire les données.

In [ ]:
# PCA analysis using FactoMineR
song_pca <- song[, c(3, 7, 9:10, 12, 14:20)]

# Perform PCA
pca <- PCA(song_pca,scale.unit = TRUE, graph = FALSE,ncp = 7,quali.sup = 2)

# Afficher le pourcentage de variance expliquée par chaque composante principale
fviz_eig(pca, addlabels = TRUE, ylim = c(0, 40))

# Calculer la variance cumulée
explained_variance <- pca$eig[, 2]  # La deuxième colonne contient le pourcentage de variance expliquée
cumulative_variance <- cumsum(explained_variance) 

# Tracer la variance cumulée
plot(cumulative_variance, xlab = "Nombre de composantes principales", ylab = "Variance cumulée", type = "b")
abline(h = 80, col = "red", lty = 2)  # Ligne horizontale à 80% de variance expliquée

**Interprétation :** 

L’analyse de la variance expliquée montre que les 7 premières composantes principales permettent de représenter 80,1 % de la variance totale du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les 11 variables numériques initiales (comme danceability, energy, speechiness, tempo, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une réduction significative de la complexité du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait pertinent de conserver ces 7 composantes principales pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’interpréter les trois premières composantes principales, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (playlist_genre).

In [ ]:
# Corrélation des variables
corrplot(pca$var$cor, is.corr = FALSE, method = "ellipse")

# Tracer les variables sur le plan factoriel dim 1-2
fviz_pca_var(pca, axes=c(1,2),col.var = "contrib", gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), repel = TRUE) +
  labs(title = "Variables sur le plan factoriel") +
  theme_minimal()

# Tracer les variables sur le plan factoriel dim 1-3
  fviz_pca_var(pca, axes = c(1, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 1-3)") +
    theme_minimal()

# Tracer les variables sur le plan factoriel dim 2-3
  fviz_pca_var(pca, axes = c(2, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 2-3)") +
    theme_minimal()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

**Composante principale 1 :**
    
Les variables energy (-0.91), loudness (-0.80) et acousticness (+0.72) sont linéairement corrélées avec la première composante principale, en soulignant que energy et loudness sont inversément corrélées avec acousticness. Cela indique que la CP1 oppose les morceaux énergiques, forts en volume et peu acoustiques (ex : rock, électro) aux morceaux calmes, acoustiques et peu énergétiques (ex : folk, classique).

**Composante principale 2 :**

Sur le graphique de gauche, on remarque une opposition des variables instrumentalness (+0.45), duration_s (+0.38) contre danceability (-0.68), valence (-0.62), track_popularity (-0.37), speechiness (-0.39).

Cette composante principale oppose deux profils de morceaux :
    D’un côté, les morceaux instrumentaux, longs et peu populaires (forte contribution de instrumentalness et duration_s), souvent associés à des genres comme le classique ou le jazz.
    De l’autre, les chansons courtes, dansantes, joyeuses et populaires (forte contribution de danceability, valence et track_popularity), typiques de la pop ou de la musique de club.
    Enfin, cette opposition suggère que les morceaux avec des paroles marquées (speechiness) et une structure rythmique engageante (danceability) sont plus susceptibles de générer de la popularité.

**Composantes principales 2 et 3 :**

Sur le graphique au centre, on peut extraire plusieurs informations :
    Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.

**Composante principale 3 :**

La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (danceability) et mood positif (valence), mais qui restent peu populaires (track_popularity). Ces morceaux sont souvent instrumentaux (instrumentalness), longs (duration_s) et à tempo faible (tempo), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.

**Remarque:** on peut noter que les valeurs ne sont pas exactement les mêmes que sur le notebook Python. L'ACP sous R ne prend pas forcément la même base que sur Python, ce qui explique les valeurs parfois négatives ou positives.

Pour mieux comprendre à quoi correspond les type morceaux extraits par ces composantes principales, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# Extraction des coordonnées des individus sur la dimension 1
dim1_coords <- pca$ind$coord[, 1]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices <- order(dim1_coords)[1:5]
max_indices <- order(dim1_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals <- song[c(min_indices, max_indices), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals$Dim1_Value <- dim1_coords[c(min_indices, max_indices)]

# Réorganiser les colonnes pour mettre Category et Dim1_Value au début
col_order <- c("Category", "Dim1_Value", colnames(extreme_individuals)[1:(ncol(extreme_individuals)-2)])
extreme_individuals <- extreme_individuals[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals)
}



Pour mieux cerner les types de morceaux représentés aux extrémités de la première composante principale (PC1), nous avons identifié les individus (chansons) ayant les contributions les plus élevées, positives comme négatives.

    Du côté des contributions positives, on retrouve majoritairement des morceaux rock, hard rock ou pop rock très énergiques et puissants tels que American Idiot (Green Day), Beauty Queen (BLVK SWVN) ou ATTENTION ATTENTION (Shinedown). Ces morceaux sont caractérisés par une énergie élevée, une forte intensité sonore (loudness) et une faible acoustique, ce qui confirme bien la structure mise en évidence par la CP1. Notons aussi This Is How We Do It (Montell Jordan), un morceau R&B énergique, qui se distingue des autres par son genre mais partage les mêmes caractéristiques acoustiques.

    À l’opposé, les morceaux à contribution très négative sur PC1 sont des titres à forte acoustique, peu énergiques et très faibles en loudness. Il s'agit notamment de sons ambiants, relaxants ou naturels, comme Peaceful Forest ou Tropical Rainforest at Dawn, mais aussi de titres R&B ou indie très doux (Small de chloe moriondo). Ces morceaux incarnent l'autre extrémité de la CP1 : des chansons calmes, acoustiques et à faible énergie, souvent issues de sous-genres comme tropical, indie poptimism ou new jack swing.

Cette opposition renforce l’interprétation de la CP1 comme un axe énergie / intensité sonore vs. calme / acoustique, pertinent pour distinguer deux grandes familles de styles musicaux dans le dataset.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 2
dim2_coords <- pca$ind$coord[, 2]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim2 <- order(dim2_coords)[1:5]
max_indices_dim2 <- order(dim2_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim2 <- song[c(min_indices_dim2, max_indices_dim2), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim2$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim2$Dim2_Value <- dim2_coords[c(min_indices_dim2, max_indices_dim2)]

# Réorganiser les colonnes pour mettre Category et Dim2_Value au début
col_order <- c("Category", "Dim2_Value", colnames(extreme_individuals_dim2)[1:(ncol(extreme_individuals_dim2)-2)])
extreme_individuals_dim2 <- extreme_individuals_dim2[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim2)
}



Côté contributions positives, on retrouve des titres principalement rap et latino, tels que Suge de DaBaby ou LAX de B0nds. Ces morceaux sont :

    courts,
    dansants (haute danceability),
    avec une valence élevée (émotion positive),
    mais également avec un certain niveau de speechiness, notamment pour les titres rap.

Ces morceaux partagent donc des caractéristiques propres aux chansons énergétiques, rythmées et populaires, souvent taillées pour le streaming, avec des formats courts, accrocheurs et directs.

À l’opposé, les morceaux ayant une forte contribution négative à PC2 sont très différents : on retrouve des paysages sonores naturels, ambiants ou instrumentaux comme Rain Forest and Tropical Beach Sound, Caribbean Thunderstorm, ou encore Battlement. Ces titres sont :

    longs,
    instrumentaux (forte instrumentalness),
    avec une faible valence et peu de parole,
    et souvent issus de sous-genres comme tropical, album rock, ou ambient.

Cela confirme l’interprétation initiale de la PC2 comme un axe opposant la musique instrumentale, longue et contemplative à une musique populaire, dansante et rythmée.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 3
dim3_coords <- pca$ind$coord[, 3]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim3 <- order(dim3_coords)[1:5]
max_indices_dim3 <- order(dim3_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim3 <- song[c(min_indices_dim3, max_indices_dim3), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim3$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim3$Dim3_Value <- dim3_coords[c(min_indices_dim3, max_indices_dim3)]

# Réorganiser les colonnes pour mettre Category et Dim3_Value au début
col_order <- c("Category", "Dim3_Value", colnames(extreme_individuals_dim3)[1:(ncol(extreme_individuals_dim3)-2)])
extreme_individuals_dim3 <- extreme_individuals_dim3[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim3)
}



La troisième composante principale met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

Du côté des contributions négatives, on retrouve des titres de pop et R&B calmes et acoustiques comme raindrops (an angel cried) (Ariana Grande) ou You Are The Reason (Calum Scott). Ces chansons ont :

    une acousticness élevée (acapela, guitare acoustique, piano),
    un tempo légèrement plus rapide que la médiane des morceaux,
    une faible énergie, mais un potentiel émotionnel fort (valence variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

Les contributions positives, quant à elles, sont largement dominées par des morceaux EDM ou latino instrumentaux, comme I Feel Love ou Chase. Ces morceaux sont :

    longs,
    très instrumentaux,
    énergiques mais souvent moins "accessibles" émotionnellement (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la PC3 oppose des créations acoustiques à forte charge émotionnelle à des morceaux instrumentaux, électroniques, longs, aux dynamiques parfois complexes ou répétitives.

On remarque le signe des contributions est inversé par rapport à l'analyse en composantes principales (ACP) réalisée sous Python.

## 2. Réduction de dimension par Analyse en Correspondances Multiples (MCA)

## Multiple Factorial Analysis (MCA)

**1. Variables catégorielles à utiliser dans la MCA**
Voici les variables qualitatives que l'on pourrait envisager d'utiliser pour la MCA sur le dataset Spotify :  
- `track_artist`  
- `track_album_name` (attention à trop de modalités rares, peut-être garder que les plus fréquentes ou ne pas inclure à cause du grand nombre d’artistes)  
- `playlist_name`  
- `playlist_genre`  
- `playlist_subgenre`  
- `key`  
- `mode`  

---

**2. Questions que la MCA peut aider à explorer**

**a) Quels sont les groupes/cluster de modalités similaires ?**
- Est-ce que certains genres et sous-genres de playlists s’associent fréquemment ?  
- Certains "modes" (majeur/minor) sont-ils plus fréquents dans certains genres ?  
- Y a-t-il des clés (`key`) musicales qui sont typiques de certains genres ou playlists ?  

La MCA permettra de représenter graphiquement (biplot) ces modalités et d’identifier des associations fortes.

**b) Est-ce que certains artistes ou playlists ont un profil qualitatif particulier ?**
- Par exemple, certains artistes seraient-ils associés à un genre et sous-genre spécifiques, ou à un mode particulier ?  
- Y a-t-il des clusters d’artistes / playlists qui partagent des caractéristiques particulières (clé, mode, genre) ?

**3. Comment interpréter la MCA ici**

- **Axes factoriels** : Chaque axe correspond à une dimension qui résume des associations fortes entre modalités. Par exemple, un axe peut opposer les genres "Rock" à "Pop", ou des clés majeures à mineures.
- **Modalités proches dans l’espace** : Modalités proches signifient qu’elles co-apparaissent souvent dans les observations (ex. certains genres + mode majeur).
- **Observation** : Si tu represents les observations (chansons) dans l’espace MCA, celles proches partagent des profils catégoriels similaires.

---

**4. Utilisations concrètes/academic use cases**

- **Profil des genres musicaux** : Comprendre quels modes/clés/sous-genres caractérisent les genres populaires sur Spotify.  
- **Segmentation qualitative des playlists** : Y a-t-il des types de playlists/musiques qui partagent des caractéristiques qualitatives communes ?  
- **Analyse de diversité** : Mesurer dans quelle mesure certains artistes/genres sont hétérogènes ou homogènes quant à leurs caractéristiques catégorielles.  
- **Préparation à une classification** : Par exemple, combiner le résultat de la PCA (variables numériques) avec la MCA (variables qualitatives) dans une analyse factorielle mixte ou pour enrichir un modèle prédictif.

---

**5. Exemple de questions précises à poser**

- Les genres musicaux sont-ils liés à certaines tonalités ou modes ?  
- Les sous-genres présents dans la même playlist sont-ils proches ou éloignés dans l’espace MCA ?  
- Y a-t-il des clés rares ou des modes minoritaires associés à certains genres uniquement ?  
- Peut-on détecter des groupes de playlists ou artistes avec des profils qualitatifs similaires ?  

---

**En conclusion**
La MCA t’aide surtout à **explorer et visualiser les relations entre variables qualitatives** et leurs modalités sur ton dataset Spotify, ce qui complète bien la PCA sur les variables numériques. C’est une étape utile pour comprendre la structure qualitative de tes données avant d’envisager une modélisation supervisée ou une analyse plus approfondie.

---

Si tu veux, je peux aussi te fournir un exemple de code Python pour réaliser une MCA avec `prince` ou en R avec `FactoMineR` sur ton dataset.

In [ ]:
#MCA avec FactoMineR sur les variables playlist_genre et mode
#song_mca conserve uniquement les variables playlist_genre et mode
song_mca <- song[, c('playlist_genre', 'mode')]
# Perform MCA
mca <- MCA(song_mca, graph = FALSE, ncp = 2)

#Tracer les variables sur le plan factoriel
fviz_mca_var(mca, col.var = "contrib", gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
             repel = TRUE) +
  labs(title = "Variables sur le plan factoriel") +
  theme_minimal()
